# Proyecto integrado 9

Hola Andrea!

Soy **Patricio Requena** 👋. Es un placer ser el revisor de tu proyecto el día de hoy!

Revisaré tu proyecto detenidamente con el objetivo de ayudarte a mejorar y perfeccionar tus habilidades. Durante mi revisión, identificaré áreas donde puedas hacer mejoras en tu código, señalando específicamente qué y cómo podrías ajustar para optimizar el rendimiento y la claridad de tu proyecto. Además, es importante para mí destacar los aspectos que has manejado excepcionalmente bien. Reconocer tus fortalezas te ayudará a entender qué técnicas y métodos están funcionando a tu favor y cómo puedes aplicarlos en futuras tareas. 

_**Recuerda que al final de este notebook encontrarás un comentario general de mi parte**_, empecemos!

Encontrarás mis comentarios dentro de cajas verdes, amarillas o rojas, ⚠️ **por favor, no muevas, modifiques o borres mis comentarios** ⚠️:


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si todo está perfecto.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si tu código está bien pero se puede mejorar o hay algún detalle que le hace falta.
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si de pronto hace falta algo o existe algún problema con tu código o conclusiones.
</div>

Puedes responderme de esta forma:
<div class="alert alert-block alert-info">
<b>Respuesta del estudiante</b> <a class=“tocSkip”></a>
</div>

El objetivo del presente proyecto es desarrollar un modelo que pueda predecir si un cliente dejará Beta Bank pronto. 

# Contenido <a id='back'></a>

* [Etapa 1. Abrir el arhivo de datos y análisis general](#data_review)
* [Etapa 2. Preparar los datos](#data_preprocessing)
* [Etapa 3. Examinar el equilibrio de clases](#class_balance)
* [Etapa 4. Mejorar la calidad del modelo](#model_quality)
* [Etapa 5. Prueba final](#final_test)
* [Etapa 6. Conclusión](#end)

<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Buen trabajo! El incluir una tabla de contenido ayuda a la navegación en tu notebook
</div>

## Etapa 1. Abrir el archivo de datos y análisis general <a id='data_review'></a>

In [1]:
#Llamar a las librerías
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle

<div class="alert alert-block alert-info">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Cuando tengas que importar varias librerías una buena práctica es seguir el siguiente órden en las mismas:

- Primero todas las librerías en orden alfabético que vienen ya con python cómo `datetime`, `os`, `json`, etc.
- Luego de las librerías de Python si las de terceros en orden alfabético cómo `pandas`, `scipy`, `numpy`, etc.
- Por último, en el caso de que armes tu propio módulo en tu proyecto esto debería ir en tercer lugar, y recuerda siempre ordenar cada tipo por orden alfabético
</div>

In [2]:
#Llamar a la base de datos
clientes = pd.read_csv('/datasets/Churn.csv')

In [3]:
#Observar información general de la data
print(clientes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
None


In [4]:
#Convertir variables
if clientes['Tenure'].isna().sum() > 0:
    clientes['Tenure'].fillna(clientes['Tenure'].median(), inplace=True)
    
clientes['Tenure'] = clientes['Tenure'].astype(int)

In [5]:
#Eliminar columnas no necesarias
clientes.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [6]:
print(clientes.head())

   CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0          619    France  Female   42       2       0.00              1   
1          608     Spain  Female   41       1   83807.86              1   
2          502    France  Female   42       8  159660.80              3   
3          699    France  Female   39       1       0.00              2   
4          850     Spain  Female   43       2  125510.82              1   

   HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0          1               1        101348.88       1  
1          0               1        112542.58       0  
2          1               0        113931.57       1  
3          0               0         93826.63       0  
4          1               1         79084.10       0  


In [7]:
print(clientes.describe())

        CreditScore           Age       Tenure        Balance  NumOfProducts  \
count  10000.000000  10000.000000  10000.00000   10000.000000   10000.000000   
mean     650.528800     38.921800      4.99790   76485.889288       1.530200   
std       96.653299     10.487806      2.76001   62397.405202       0.581654   
min      350.000000     18.000000      0.00000       0.000000       1.000000   
25%      584.000000     32.000000      3.00000       0.000000       1.000000   
50%      652.000000     37.000000      5.00000   97198.540000       1.000000   
75%      718.000000     44.000000      7.00000  127644.240000       2.000000   
max      850.000000     92.000000     10.00000  250898.090000       4.000000   

         HasCrCard  IsActiveMember  EstimatedSalary        Exited  
count  10000.00000    10000.000000     10000.000000  10000.000000  
mean       0.70550        0.515100    100090.239881      0.203700  
std        0.45584        0.499797     57510.492818      0.402769  
min    

In [8]:
#Ver si hay duplicados
print(clientes.duplicated().sum())

0


### Conclusión etapa 1
Pudo observarse que en la data **clientes** hay valores perdidos en la variable *Tenure*, por ello se decidió reemplazar estos valores perdidos con el valor de la mediana. Sumado a lo anterior, se borraron tres variables de la data, debido a que estas variables pueden no ser útiles para la predicción. Además, a partir del valor de la desviación estándar, es posible concluir que la mitad de variables tienen datos atípicos (CreditScore, Age, Balance y EstimatedSalary). Finalmente, no se observan duplicados. 

<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Carga de datos y exploración inicial correcta!
</div>

## Etapa 2. Preparar los datos <a id='data_preprocessing'></a>

In [9]:
#Codificación de variables categóricas
clientes = pd.get_dummies(clientes,columns=['Geography', 'Gender'], drop_first=True)

In [10]:
#División de datos
train_data, test_data = train_test_split(clientes, test_size=0.3, random_state=12345)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=12345)

In [11]:
#Variables de características y objetivo para cada conjunto
features_train = train_data.drop('Exited', axis=1)
target_train = train_data['Exited']

features_valid = val_data.drop('Exited', axis=1)
target_valid = val_data['Exited']

features_test = test_data.drop('Exited', axis=1)
target_test = test_data['Exited']

In [12]:
# Escalado de características
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

scaler = StandardScaler()

train_data = train_data.copy()  
val_data = val_data.copy()      
test_data = test_data.copy()

train_data[numeric] = scaler.fit_transform(train_data[numeric])
val_data[numeric] = scaler.transform(val_data[numeric])
test_data[numeric] = scaler.transform(test_data[numeric])

### Conclusión etapa 2
Primero se codificaron las variables categóricas no dummies en dummies, con el objetivo de hacerlas compatibles con el formato que requieren los modelos de machine learning. Luego se crearon tres conjuntos de datos, siendo estos de entrenamiento, de prueba y de validación respectivamente. Finalmente se estandarizaron las características numéricas del modelo. 

## Etapa 3. Examinar el equilibrio de clases <a id='class_balance'></a>

In [13]:
#Distribución de clases en el conjunto de entrenamiento
class_distribution = train_data['Exited'].value_counts(normalize=True)
print(class_distribution)

0    0.798429
1    0.201571
Name: Exited, dtype: float64


Es posible observar que existe un fuerte desequilibrio. Lo anterior debido a que los clientes que permanecen en el banco representan al 79.24% de los datos, mientras que los clientes que se van representan al 20.76%. 

In [14]:
#Definir los mejores puntajes
best_f1_score = 0
best_depth = 0
best_auc_roc = 0
best_min_samples_split = 0
best_min_samples_leaf = 0

#Bucle para encontrar los mejores hiperparámetros
for depth in range(1, 11):
    for min_samples_split in [2, 5, 10]:
        for min_samples_leaf in [1, 2, 4]:
            model = RandomForestClassifier(
                random_state=12345,
                max_depth=depth,
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf,
                n_estimators=50, 
                criterion='gini'
            )
            model.fit(features_train, target_train)
            predictions_valid = model.predict(features_valid)

            #Calcular F1 Score y AUC-ROC
            f1 = f1_score(target_valid, predictions_valid)
            probabilities_valid = model.predict_proba(features_valid)[:, 1]
            auc_roc = roc_auc_score(target_valid, probabilities_valid)

            #Actualizar mejores puntajes si el nuevo es mejor
            if f1 > best_f1_score:
                best_f1_score = f1
                best_depth = depth
                best_auc_roc = auc_roc
                best_min_samples_split = min_samples_split
                best_min_samples_leaf = min_samples_leaf

#Imprimir los mejores resultados
print(f"Mejor profundidad: {best_depth}")
print(f"Mejor min_samples_split: {best_min_samples_split}")
print(f"Mejor min_samples_leaf: {best_min_samples_leaf}")

Mejor profundidad: 8
Mejor min_samples_split: 2
Mejor min_samples_leaf: 2


In [15]:
#Predicciones en el conjunto de validación
predictions = model.predict(features_valid)

In [16]:
#Calcular la matriz de confusión
cm = confusion_matrix(target_valid, predictions)
print('Matriz de Confusión:\n', cm)

Matriz de Confusión:
 [[1165   28]
 [ 168  139]]


Puede observarse que el modelo predice correctamente 1165 clientes que no se irán, así como 139 clientes que se irán. En contraste, predice de manera incorrecta 28 clientes que se irán, pero no lo hicieron y 168 que no se irán, pero lo hicieron. Por tanto, es posible afirmar que el modelo realiza un buen trabajo clasificando los ejemplos negativos, sin embargo, presenta algunas dificultades con los ejemplos positivos.

In [17]:
#Calcular precisión, recall y valor F1
precision = precision_score(target_valid, predictions)
recall = recall_score(target_valid, predictions)
f1 = f1_score(target_valid, predictions)

print(f'Precisión: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'Valor F1: {f1:.2f}')

Precisión: 0.83
Recall: 0.45
Valor F1: 0.59


La precisión indica que el 83% de las veces que el modelo predijo que un cliente se iría, estaba en lo correcto. Adicionalmente, el recall muestra que se identifica de manera correcta al 45% de los clientes que realmente se irán. Sumado a lo anterior, el F1 sugiere que el modelo tiene un rendimiento moderado en general, pero todavía queda espacio de mejora.

### Conclusión etapa 3
Con la información precedente es posible afirmar que existe un gran desequilibrio de clases, debido a que 8 de cada 10 clientes se quedan en el banco, mientras que solo se van 2 de cada 10. Lo anterior podría generar que el modelo prediga más la clase mayoritaria (0), ignorando a la minoritaria (1).

<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Buen trabajo con el entrenamiento sin balancear los datos, al estar los datos en desbalance es normal que en primera instancia las métricas sean bajas y por eso el accuracy en este caso no es la más recomendable ya que si el modelo predice siempre la clase mayoritaria el accuracy será alto. Por eso es mejor usar f1-score
</div>

## Etapa 4. Mejorar la calidad del modelo <a id='model_quality'></a>

In [18]:
#Sobremuestreo
def upsample (features, target, repeat):
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]
    
    features_upsampled = pd.concat([features_zeros]+[features_ones]*repeat)
    target_upsampled = pd.concat([target_zeros]+[target_ones]*repeat)
    
    features_upsampled, target_upsampled = shuffle(
    features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

#Aplicar sobremuestreo
features_upsampled, target_upsampled = upsample(features_train, target_train, 10)

#Definir los mejores puntajes
best_f1_score = 0
best_params = {}

#Bucle para encontrar los mejores hiperparámetros
for depth in [5, 7, 10]: 
    for min_samples_split in [2, 3, 5]:  
        for min_samples_leaf in [1, 2, 5]: 
            for n_estimators in [10, 30, 50, 100, 150]:  
                for criterion in ['gini', 'entropy']:
                    model = RandomForestClassifier(
                        random_state=12345,
                        max_depth=depth,
                        min_samples_split=min_samples_split,
                        min_samples_leaf=min_samples_leaf,
                        n_estimators=n_estimators,
                        criterion=criterion
                )
                model.fit(features_upsampled, target_upsampled)
                predicted_valid = model.predict(features_valid)
                score = f1_score(target_valid, predicted_valid)
                if score > best_f1_score:
                    best_f1_score = score
                    best_params = {
                        'max_depth': depth,
                        'min_samples_split': min_samples_split,
                        'min_samples_leaf': min_samples_leaf,
                        'n_estimators': n_estimators,
                        'criterion': criterion 
                    }

#Mostrar los mejores hiperparámetros y su F1 Score
print("Mejores hiperparámetros:")
print(f"Profundidad máxima: {best_params['max_depth']}")
print(f"Mínimas muestras por división: {best_params['min_samples_split']}")
print(f"Mínimas muestras por hoja: {best_params['min_samples_leaf']}")
print(f"Mejor número de árboles de decisión: {best_params['n_estimators']}")
print(f"Criterio: {best_params['criterion']}") 
print(f"Mejor F1 Score: {best_f1_score}")

#Calcular AUC-ROC con los mejores hiperparámetros
best_model = RandomForestClassifier(
    random_state=12345,
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    n_estimators=best_params['n_estimators'],
    criterion=best_params['criterion'] 
)
best_model.fit(features_upsampled, target_upsampled)
probabilities_valid = best_model.predict_proba(features_valid)[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_valid)
print('Mejor AUC-ROC:', auc_roc)

Mejores hiperparámetros:
Profundidad máxima: 10
Mínimas muestras por división: 2
Mínimas muestras por hoja: 1
Mejor número de árboles de decisión: 10
Criterio: entropy
Mejor F1 Score: 0.5485592315901814
Mejor AUC-ROC: 0.8379717734559087


Replicar los casos de clientes que se fueron en un modelo de árbol aleatorio con 10 de profundidad, que requiere al menos 2 muestras para cada división, así como 1 muestra para cada hoja y con 10 árboles otorgó los siguientes resultados:

1. El modelo logra un equilibrio razonable entre precisión y recall (0.55).
2. El modelo tiene buena capacidad para discriminar entre las clases positivas y negativas (0.84)

In [19]:
#Submuestreo
def downsample (features, target, fraction):
    features_zeros = features_train[target_train == 0]
    features_ones = features_train[target_train == 1]
    target_zeros = target_train[target_train == 0]
    target_ones = target_train[target_train == 1]
    
    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)]+[features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)]+[target_ones])
    
    features_downsampled, target_downsampled = shuffle(
    features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

#Aplicar submuestreo
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.1)

# Definir los mejores puntajes
best_f1_score = 0
best_params = {}

#Bucle para encontrar los mejores hiperparámetros
for depth in [5, 7, 10]: 
    for min_samples_split in [2, 3, 5]:  
        for min_samples_leaf in [1, 2, 5]: 
            for n_estimators in [10, 30, 50, 100, 150]: 
                for criterion in ['gini', 'entropy']:
                    model = RandomForestClassifier(
                        random_state=12345,
                        max_depth=depth,
                        min_samples_split=min_samples_split,
                        min_samples_leaf=min_samples_leaf,
                        n_estimators=n_estimators,
                        criterion=criterion
                )
                model.fit(features_downsampled, target_downsampled)
                predicted_valid = model.predict(features_valid)
                score = f1_score(target_valid, predicted_valid)
                if score > best_f1_score:
                    best_f1_score = score
                    best_params = {
                        'max_depth': depth,
                        'min_samples_split': min_samples_split,
                        'min_samples_leaf': min_samples_leaf,
                        'n_estimators': n_estimators,
                        'criterion': criterion
                    }

#Mostrar los mejores hiperparámetros y su F1 Score
print("Mejores hiperparámetros:")
print(f"Profundidad máxima: {best_params['max_depth']}")
print(f"Mínimas muestras por división: {best_params['min_samples_split']}")
print(f"Mínimas muestras por hoja: {best_params['min_samples_leaf']}")
print(f"Mejor número de árboles de decisión: {best_params['n_estimators']}")
print(f"Criterio: {best_params['criterion']}") 
print(f"Mejor F1 Score: {best_f1_score}")

#Calcular AUC-ROC con los mejores hiperparámetros
best_model = RandomForestClassifier(
    random_state=12345,
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    n_estimators=best_params['n_estimators'],
    criterion=best_params['criterion'] 
)
best_model.fit(features_downsampled, target_downsampled)
probabilities_valid = best_model.predict_proba(features_valid)[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_valid)
print('Mejor AUC-ROC:', auc_roc)

Mejores hiperparámetros:
Profundidad máxima: 10
Mínimas muestras por división: 5
Mínimas muestras por hoja: 1
Mejor número de árboles de decisión: 150
Criterio: entropy
Mejor F1 Score: 0.4677685950413223
Mejor AUC-ROC: 0.8435253419103291


Reducir los casos de clientes que se quedan en un modelo de árbol aleatorio con 10 de profundidad, que requiere al menos 5 muestras para cada división, así como 1 muestra para cada hoja y con 150 árboles otorgó los siguientes resultados:

1. El modelo logra un equilibrio razonable entre precisión y recall. Sin embargo, este es menor al obtenido en el modelo de sobremuestreo (0.47).
2. El modelo tiene buena capacidad para discriminar entre las clases positivas y negativas. Siendo este ligeramente mayor al obtenido en el modelo de sobremuestreo (0.84).

<div class="alert alert-block alert-danger">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Te recomiendo probar otros modelos como `DecisionTreeClassifier` y que pruebas aplicando sobremuestreo y submuestreo para que compares el f1-score que te devuelven, recuerda que para aprobar debes obtener mínimo 0.59 de f1-score
</div>

In [23]:
# Definir el rango de hiperparámetros
depth_range = range(1, 11) 
min_samples_split_range = [2, 3, 5] 
min_samples_leaf_range = [1, 2, 5]  
n_estimators_range = [10, 30, 50, 100]
criteria = ['gini', 'entropy'] 

#Inicializar variables para guardar el mejor modelo
best_score = 0
best_params = {}

#Bucle para encontrar los mejores hiperparámetros
for depth in depth_range:
    for min_samples_split in min_samples_split_range:
        for min_samples_leaf in min_samples_leaf_range:
            for n_estimators in n_estimators_range:
                for criterion in criteria:
                    #Crear y entrenar el modelo de RandomForestClassifier
                    model = RandomForestClassifier(
                        random_state=12345,
                        max_depth=depth,
                        min_samples_split=min_samples_split,
                        min_samples_leaf=min_samples_leaf,
                        n_estimators=n_estimators,
                        criterion=criterion
                    )
                    
                    #Ajustar el modelo con los datos de entrenamiento
                    model.fit(features_train, target_train)
                    
                    #Predecir las etiquetas para los datos de validación
                    predicted_valid = model.predict(features_valid)
                    
                    #Calcular el F1 Score
                    score = f1_score(target_valid, predicted_valid)
                    
                    # Verificar si el F1 Score es el mejor
                    if score > best_score:
                        best_score = score
                        best_params = {
                            'max_depth': depth,
                            'min_samples_split': min_samples_split,
                            'min_samples_leaf': min_samples_leaf,
                            'n_estimators': n_estimators,
                            'criterion': criterion
                        }

#Mostrar los mejores hiperparámetros y su F1 Score
print("Mejores hiperparámetros:")
print(f"Profundidad máxima: {best_params['max_depth']}")
print(f"Mínimas muestras por división: {best_params['min_samples_split']}")
print(f"Mínimas muestras por hoja: {best_params['min_samples_leaf']}")
print(f"Mejor número de árboles de decisión: {best_params['n_estimators']}")
print(f"Criterio: {best_params['criterion']}")
print(f"Mejor F1 Score: {best_score}")

#Calcular AUC-ROC con los mejores hiperparámetros
best_model = RandomForestClassifier(
    random_state=12345,
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    n_estimators=best_params['n_estimators'],
    criterion=best_params['criterion']
)
best_model.fit(features_train, target_train)
probabilities_valid = best_model.predict_proba(features_valid)[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_valid)
print('Mejor AUC-ROC:', auc_roc)

Mejores hiperparámetros:
Profundidad máxima: 9
Mínimas muestras por división: 3
Mínimas muestras por hoja: 1
Mejor número de árboles de decisión: 10
Criterio: gini
Mejor F1 Score: 0.5978947368421053
Mejor AUC-ROC: 0.8460263589723986


Ajustar el peso en el árbol de decisión y considerando 9 de profundidad, 3 muestras mínimas por división, 1 muestra mínima por hoja, 10 árboles y el criterio Gini para medir la calidad de las divisiones brinda los siguientes resultados:

1. El modelo tiene un buen equilibrio entre falsos positivos y falsos negativos (0.60).
2. El modelo tiene un muy buen rendimiento para discriminar entre las clases positivas y negativas.

In [27]:
#Definir el modelo de bosque aleatorio con los mejores hiperparámetros
best_model = RandomForestClassifier(
    random_state=12345,
    max_depth=9, 
    min_samples_split=3,  
    min_samples_leaf=1,  
    n_estimators=10, 
    criterion='gini' 
)

#Entrenar el modelo con los datos de entrenamiento
best_model.fit(features_train, target_train)

#Predecir las probabilidades para los datos de validación
probabilities_valid = best_model.predict_proba(features_valid)[:, 1]

#Calcular AUC-ROC
auc_roc = roc_auc_score(target_valid, probabilities_valid)
print('AUC-ROC:', auc_roc)

#Encontrar el mejor umbral basado en el F1 Score
best_threshold = 0
best_f1 = 0

#Probar una variedad de umbrales para maximizar el F1 Score
for threshold in np.arange(0, 0.3, 0.02):
    predicted_valid = probabilities_valid > threshold
    f1 = f1_score(target_valid, predicted_valid)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold
    print(f'Umbral = {threshold:.2f} | F1 Score = {f1:.3f}')

print(f'Mejor umbral = {best_threshold:.2f} | Mejor F1 Score = {best_f1:.3f}')

#Predecir con el mejor umbral
predicted_valid_best_threshold = probabilities_valid > best_threshold
final_f1 = f1_score(target_valid, predicted_valid_best_threshold)
print(f'F1 Score con el mejor umbral = {final_f1:.3f}')

AUC-ROC: 0.8460263589723986
Umbral = 0.00 | F1 Score = 0.340
Umbral = 0.02 | F1 Score = 0.355
Umbral = 0.04 | F1 Score = 0.390
Umbral = 0.06 | F1 Score = 0.416
Umbral = 0.08 | F1 Score = 0.444
Umbral = 0.10 | F1 Score = 0.468
Umbral = 0.12 | F1 Score = 0.500
Umbral = 0.14 | F1 Score = 0.529
Umbral = 0.16 | F1 Score = 0.551
Umbral = 0.18 | F1 Score = 0.577
Umbral = 0.20 | F1 Score = 0.587
Umbral = 0.22 | F1 Score = 0.602
Umbral = 0.24 | F1 Score = 0.605
Umbral = 0.26 | F1 Score = 0.613
Umbral = 0.28 | F1 Score = 0.625
Mejor umbral = 0.28 | Mejor F1 Score = 0.625
F1 Score con el mejor umbral = 0.625


Lo anterior nos indica que el mejor umbral sería 0.28, dado que el F1 Score en este caso es de 0.625. 

### Conclusión etapa 4
En base a la información precedente, lo mejor sería ajustar el peso de clases en el modelo de árboles de decisión. Lo anterior debido a que este método nos da el mejor F1 Score y el mejor valor de AUC-ROC. 

## Etapa 5. Prueba final <a id='final_test'></a>

In [30]:
#Definir el modelo de bosque aleatorio con los mejores hiperparámetros
final_model = RandomForestClassifier(
    random_state=12345,
    max_depth=9,  
    min_samples_split=3,  
    min_samples_leaf=1,  
    n_estimators=10, 
    criterion='gini' 
)

#Entrenar el modelo con el conjunto de entrenamiento completo
final_model.fit(features_train, target_train)

#Predecir las probabilidades en el conjunto de prueba
probabilities_test = final_model.predict_proba(features_test)[:, 1]

#Usar el umbral de 0.28 para predecir
best_threshold = 0.28
predicted_test_best_threshold = probabilities_test > best_threshold

#Evaluar el modelo en el conjunto de prueba
final_f1_test = f1_score(target_test, predicted_test_best_threshold)
final_auc_roc_test = roc_auc_score(target_test, probabilities_test)

print(f'F1 Score final en el conjunto de prueba = {final_f1_test:.3f}')
print(f'AUC-ROC final en el conjunto de prueba = {final_auc_roc_test:.3f}')

F1 Score final en el conjunto de prueba = 0.614
AUC-ROC final en el conjunto de prueba = 0.851


Lo anterior indica que el modelo tiene un muy buen rendimiento en lo que se refiere a la precisión y al recall en el conjunto de prueba (0.614) y que es muy capaz de discriminar entre las clases positiva y negativa (0.851). 

### Conclusión etapa 5
Se destaca la mejora en el F1 Score a comparación del modelo sin el ajuste de peso de las clases y que la probabilidad de que el modelo asigne una puntación más alta a un ejemplo positivo que a uno negativo es del 73.4%.

## Etapa 6. Conclusión <a id='end'></a>

En base a lo anterior es posible concluir que el mejor modelo para analizar este caso es el árbo ajustando el peso de las clases. Además, es posible afirmar que el modelo puede discriminar de manera adecuada entre los clientes que abandonan el banco y los que no lo hacen.

<div class="alert alert-block alert-info">
<b>Comentario general (1ra Iteracion)</b> <a class=“tocSkip”></a>

Muy buen proyecto Andrea, se nota tu entendimiento sobre las distintas técnicas que se pueden usar para lidiar con un dataset desbalanceado pero puedes probar con otros modelos como los basados en árboles para ver el desempeño que tienen en la clasificación, ya que LogisticRegression no logró superar el umbral requerido para este proyecto que es de 0.59 en f1 score.
</div>

<div class="alert alert-block alert-success">
<b>Comentario del revisor (2da Iteracion)</b> <a class=“tocSkip”></a>

Perfecto! Ahora obtuviste un modelo que superó el umbral requerido para aprobar el proyecto, por lo general los modelos basados en árboles manejan mejor los datasets desbalanceados.
    
Como recomendación para tus próximos proyectos de machine learning es que en tus conclusiones redactes tu interpretación de por que los modelos dan diferentes resultados, como afectaron los datos al desempeño de los mismos, y demás temas relacionados.
    
Saludos!
</div>